## 폴더 구조
### 기존 폴더구조
```
 Sample
 └ 2d_label
 └ label
 └ calib
 └ camera
 └ lidar
 └ (points): npy
 └ (velodyne): bin
```
    
### 변경 폴더구조
```
custom
└ ImageSets
└ points
└ labels

# read info of json

In [32]:
import json
import os
from glob import glob

In [33]:
DATASET_PATH = "/data/NIA50/OpenPCD/data/2-050_sensor_sample"
LABEL = "2d_label"
label_files = sorted(glob(DATASET_PATH + "/" + LABEL + "/*.json"))

In [34]:
with open(label_files[0], "r") as f:
    label = json.load(f)
    
print(label["objects"][:2])

[{'id': 17, 'class': 'Medium_Truck', 'front': [{'x': 543, 'y': 815}, {'x': 587, 'y': 818}, {'x': 585, 'y': 890}, {'x': 541, 'y': 890}], 'back': [{'x': 706, 'y': 802}, {'x': 748, 'y': 806}, {'x': 746, 'y': 886}, {'x': 704, 'y': 886}]}, {'id': 14, 'class': 'Truck', 'front': [{'x': 972, 'y': 811}, {'x': 995, 'y': 817}, {'x': 994, 'y': 898}, {'x': 970, 'y': 898}], 'back': [{'x': 1302, 'y': 892}, {'x': 1303, 'y': 798}, {'x': 1304, 'y': 805}, {'x': 1303, 'y': 893}]}]


In [35]:
from collections import defaultdict, Counter
categories = set()
category_counter = Counter()
for label_file in label_files:
    with open(label_file, "r") as f:
        label = json.load(f)
    for l in label["objects"]:
        categories.add(l['class'])
        category_counter[l['class']] += 1

In [36]:
print(categories)
print(category_counter)

{'Truck', 'truck', 'Medium_Truck', 'bus', 'Bus', 'Car', 'box-svg-selected', 'Adult', 'SUV', 'car'}
Counter({'car': 743, 'truck': 435, 'Car': 221, 'Adult': 221, 'bus': 149, 'box-svg-selected': 61, 'Truck': 57, 'SUV': 54, 'Medium_Truck': 53, 'Bus': 52})


# change cagetory 'car' to 'Car'

In [69]:
for lable_file in label_files:
    with open(label_file, "r") as f:
        label = json.load(f)
    for l in label["objects"]:
        if l['class'] == 'bus':
            l['class'] = 'Bus'
    
    label = json.dumps(label)
    with open(os.path.join(DATASET_PATH, LABEL, lable_file), "w") as f:
        f.write(label)

In [ ]:
# convert pcd to bin
# pip3 install --upgrade git+https://github.com/klintan/pypcd.git
!python pcd2bin.py --pcd_path /data/NIA50/OpenPCD/data/2-050_sensor_sample/lidar --bin_path /data/NIA50/OpenPCD/data/2-050_sensor_sample/velodyn


# Custom Dataset Tutorial
For the custom dataset template, we only consider the basic scenario: raw point clouds and 
their corresponding annotations. Point clouds are supposed to be stored in `.npy` format.

## Label format
We only consider the most basic information -- category and bounding box in the label template.
Annotations are stored in the `.txt`. Each line represents a box in a given scene as below:
```
# format: [x y z dx dy dz heading_angle category_name]
1.50 1.46 0.10 5.12 1.85 4.13 1.56 Vehicle
5.54 0.57 0.41 1.08 0.74 1.95 1.57 Pedestrian
```
The box should in the unified 3D box definition (see [README](../README.md))

## Files structure
Files should be placed as the following folder structure:
```
OpenPCDet
├── data
│   ├── custom
│   │   │── ImageSets
│   │   │   │── train.txt
│   │   │   │── val.txt
│   │   │── points
│   │   │   │── 000000.npy
│   │   │   │── 999999.npy
│   │   │── labels
│   │   │   │── 000000.txt
│   │   │   │── 999999.txt
├── pcdet
├── tools
```
Dataset splits need to be pre-defined and placed in `ImageSets`

## Hyper-parameters Configurations

### Point cloud features
Modify following configurations in `custom_dataset.yaml` to 
suit your own point clouds.
```yaml
POINT_FEATURE_ENCODING: {
    encoding_type: absolute_coordinates_encoding,
    used_feature_list: ['x', 'y', 'z', 'intensity'],
    src_feature_list: ['x', 'y', 'z', 'intensity'],
}
...
# In gt_sampling data augmentation
NUM_POINT_FEATURES: 4

```

#### Point cloud range and voxel sizes
For voxel based detectors such as SECOND, PV-RCNN and CenterPoint, the point cloud range and voxel size should follow:
1. Point cloud range along z-axis / voxel_size is 40
2. Point cloud range along x&y-axis / voxel_size is the multiple of 16.

Notice that the second rule also suit pillar based detectors such as PointPillar and CenterPoint-Pillar.

### Category names and anchor sizes
Category names and anchor size are need to be adapted to custom datasets.
 ```yaml
CLASS_NAMES: ['Vehicle', 'Pedestrian', 'Cyclist']  
...
MAP_CLASS_TO_KITTI: {
    'Vehicle': 'Car',
    'Pedestrian': 'Pedestrian',
    'Cyclist': 'Cyclist',
}
...
'anchor_sizes': [[3.9, 1.6, 1.56]],
...
# In gt sampling data augmentation
PREPARE: {
 filter_by_min_points: ['Vehicle:5', 'Pedestrian:5', 'Cyclist:5'],
 filter_by_difficulty: [-1],
}
SAMPLE_GROUPS: ['Vehicle:20','Pedestrian:15', 'Cyclist:15']
...
 ```
In addition, please also modify the default category names for creating infos in `custom_dataset.py`
```
create_custom_infos(
    dataset_cfg=dataset_cfg,
    class_names=['Vehicle', 'Pedestrian', 'Cyclist'],
    data_path=ROOT_DIR / 'data' / 'custom',
    save_path=ROOT_DIR / 'data' / 'custom',
)
```


## Create data info
Generate the data infos by running the following command:
```shell
python -m pcdet.datasets.custom.custom_dataset create_custom_infos tools/cfgs/dataset_configs/custom_dataset.yaml
```


## Evaluation
Here, we only provide an implementation for KITTI stype evaluation.
The category mapping between custom dataset and KITTI need to be defined 
in the `custom_dataset.yaml`
```yaml
MAP_CLASS_TO_KITTI: {
    'Vehicle': 'Car',
    'Pedestrian': 'Pedestrian',
    'Cyclist': 'Cyclist',
}
```

# Convert 'pcd' file to 'npy'

In [18]:
from pypcd import pypcd
from glob import glob
import numpy as np

# convert pcd to npy

LIDAR_PATH = "/data/NIA50/OpenPCD/data/2-050_sensor_sample/lidar"
NPY_PATY = "/data/NIA50/OpenPCD/data/2-050_sensor_sample/points"

lidar_list = sorted(glob(LIDAR_PATH+"/*.pcd"))
for lidar in lidar_list:
    pc = pypcd.PointCloud.from_path(lidar)
    points = np.vstack((pc.pc_data['x'], pc.pc_data['y'], pc.pc_data['z'])).transpose()
    np.save(NPY_PATY+"/"+lidar.split("/")[-1][:-4]+".npy", points)

# create ImageSets

In [8]:
from glob import glob
import os
from sklearn.model_selection import train_test_split

LIDAR_PATH = "/data/NIA50/OpenPCD/data/2-050_sensor_sample/lidar"
lidar_list = sorted(glob(LIDAR_PATH+"/*.pcd"))

filenames = [x[:-4] for x in sorted(os.listdir(LIDAR_PATH))]
trainset, valset = train_test_split(filenames, test_size=0.8, random_state=42)
print(trainset[:10])
print(valset[:10])

['1639543832.078959465', '1639543834.178447723', '1639543829.476884127', '1639543828.678121090', '1639543825.876186371', '1639543830.974149466', '1639543827.877243519', '1639543825.978711843', '1639543828.077119112', '1639543834.481409550']
['1639543834.076455832', '1639543831.076159239', '1639543832.775555849', '1639543830.277726889', '1639543830.175972700', '1639543829.678906441', '1639543827.978765488', '1639543833.777890205', '1639543826.776987076', '1639543825.777566671']


In [11]:
# write file names on train.txt and val.txt each line
with open("/data/NIA50/OpenPCD/data/custom/ImageSets/train.txt", "w") as f:
    f.write("\n".join(trainset))
with open("/data/NIA50/OpenPCD/data/custom/ImageSets/val.txt", "w") as f:
    f.write("\n".join(valset))

# Convert Sample 3d Label to Custom Label
## Label format

label has the most information -- category and bounding box in the label template. Annotations are stored in the `.txt`. Each line represents a box in a given scene as below:

```
# format: [x y z dx dy dz heading_angle category_name]
1.50 1.46 0.10 5.12 1.85 4.13 1.56 Vehicle
5.54 0.57 0.41 1.08 0.74 1.95 1.57 Pedestrian
```

In [42]:
import json, os
from glob import glob
from math import radians
from tqdm import tqdm
LABEL_PATH = "/data/NIA50/OpenPCD/data/2-050_sensor_sample/label"
LABEL_LIST = sorted(glob(LABEL_PATH+"/*.json"))
CUSTOM_PATH = "/data/NIA50/OpenPCD/data/custom/labels"
# {'Truck', 'truck', 'Medium_Truck', 'bus', 'Bus', 'Car', 'box-svg-selected', 'Adult', 'SUV', 'car'}

CATEGORY_CONVERT = {'Truck': 'Car', 'truck': 'Car', 
                    'Medium_Truck': 'Car', 'Van': 'Car',
                    'bus': 'Car', 'Bus': 'Car', 
                    'Car': 'Car', 'box-svg-selected': 'Car', 
                    'Adult': 'Pedestrian', 'SUV': 'Car', 
                    'car': 'Car'}


for lable in tqdm(LABEL_LIST):
    label_info = []
    with open(lable, "r") as f:
        label = json.load(f)
    
    for lb in label:
        x = lb['Value'][0]
        y = lb['Value'][1]
        z = lb['Value'][2]
        xd = lb['y']
        yd = lb['x']
        zd = lb['z']
        heading_angle = radians(lb['Heading'])
        category_name = CATEGORY_CONVERT[lb['Category']]
        
        label_format = f'{x:.4f} {y:.4f} {z:.4f} {xd:.4f} {yd:.4f} {zd:.4f} {heading_angle:.4f} {category_name}'
        label_info.append(label_format)
    
    with open(CUSTOM_PATH + "/" + lable.split("/")[-1][:-5] + ".txt", "w") as f:
        f.write("\n".join(label_info))
    

100%|██████████| 100/100 [00:00<00:00, 422.48it/s]


In [14]:
label[0]
# x: length, y: width, z: height
# Value[0]: y, Value[1]: x, Value[2]: z
# Heading: yaw(-pi ~ pi)

{'id': '1',
 'Category': 'Car',
 'Heading': -91.43391171464395,
 'Type': 'bbox',
 'Value': [-0.41179255740374865, 4.658788168227565, -1.3359166681766508],
 'x': 2.787668649899378,
 'y': 1.6241728810973413,
 'z': 0.9390174746513367,
 'Attribute': 'none'}

In [23]:
label_format

['-0.4118 4.6588 -1.3359 2.7877 1.6242 0.9390 -1.5958 Car']

# Create Custom Dataset Infomation

`python -m pcdet.datasets.custom.custom_dataset create_custom_infos tools/cfgs/dataset_configs/custom_dataset.yaml`

# Testing

- To test all the saved checkpoints of a specific training setting and draw the performance curve on the Tensorboard, add the --eval_all argument:<br>
`python test.py --cfg_file ${CONFIG_FILE} --batch_size ${BATCH_SIZE} --eval_all`

- Test with pretrained model

`python test.py --cfg_file ${CONFIG_FILE} --batch_size ${BATCH_SIZE} --ckpt ${CKPT}`
